In [13]:
from __future__ import division
import pymongo
import pandas as pd
import numpy as np
import csv
import operator

In [2]:
client = pymongo.MongoClient()
db = client.EnRoute
collection = db.ETMS2013_core2core

In [26]:
a = collection.aggregate([{'$sample':{'size': 100}}])
OD_Pairs = [['ORD','DCA'],['DCA','ORD'],['LAX','JFK'],['JFK','LAX'],
            ['FLL','JFK'],['JFK','FLL'],['IAH','BOS'],['BOS','IAH']]
for item in a:
    pair = [str(item['DEPT_APRT']), str(item['ARR_APRT'])]
    if pair not in OD_Pairs:
        OD_Pairs.append(pair)
    else:
        pass

In [27]:
len(OD_Pairs)

103

In [60]:
with open('Selected_OD3.csv','wb') as csvfile:
    writer = csv.writer(csvfile)
    for pair in OD_Pairs:
        writer.writerow(pair)

In [14]:
def MeanEff(Ori,Des):
    Ori_Des = collection.find({'DEPT_APRT':Ori,'ARR_APRT':Des,'D40_A100_LF_ACT_DIST': {'$nin':['','0']},
                                'D40_A100_DIR_DIST': {'$nin':['','0']},'D40_A100_ACH_DIST': {'$nin':['','0']}})
    Overall_Eff = []
    for doc in Ori_Des:
        Eff1 = (doc['D40_A100_ACT_DIST'] - float(doc['D40_A100_ACH_DIST']))/float(doc['D40_A100_ACH_DIST'])
        Overall_Eff.append(Eff1)
    AvgEff = sum(Overall_Eff)/len(Overall_Eff)
    return AvgEff

In [15]:
AvgEff = {}
with open('Selected_OD.csv','r') as csvfile:
    line = csv.reader(csvfile)
    for row in line:
        try:
            AvgEff[(row[0],row[1])] = MeanEff(row[0],row[1])
        except:
            print(row[0],row[1])

('LAX', 'SAN')
('PDX', 'SEA')


In [56]:
AvgEff.values()

[0.02567859276665726,
 0.02448459891375114,
 0.05785676366783496,
 0.03353123246912871,
 0.04393273346940192,
 0.03575372244941215,
 0.021002803503337466,
 0.0391483133655721,
 0.0415457268646153,
 0.03811156042338974,
 0.058012699599350966,
 0.016986996842812005,
 0.046189047603391535,
 0.02484527728312672,
 0.028693719375144085,
 0.015387156378576534,
 0.07359035691441543,
 0.012023522854289068,
 0.061839250399270865,
 0.09211910032425456,
 0.009162049020775536,
 0.036489017662234786,
 0.011138972846764992,
 0.01700672286670222,
 0.04352586817581191,
 0.021132626294282145,
 0.0816030961720591,
 0.01273202976300815,
 0.025405794307413698,
 0.04302951830885388,
 0.019297501812551635,
 0.021325332688829295,
 0.01706214837279655,
 0.02156278771722877,
 0.031649541847817576,
 0.14858758737921643,
 0.01599819604724853,
 0.016353527633782283,
 0.05628959974758388,
 0.03919377312956064,
 0.021264719648593246,
 0.01732245072800154,
 0.01888778584825351,
 0.021004192446496327,
 0.0418838647949

In [45]:
sorted_Eff = sorted(AvgEff.items(), key=operator.itemgetter(1), reverse=True)

In [53]:
sorted_Eff

[(('IAH', 'DFW'), 0.14858758737921643),
 (('CLE', 'BWI'), 0.09838121438080114),
 (('CLE', 'IAD'), 0.09211910032425456),
 (('CLT', 'BWI'), 0.0816030961720591),
 (('RDU', 'ORD'), 0.08040456251577277),
 (('STL', 'DTW'), 0.07369310185593023),
 (('ORD', 'RDU'), 0.07359035691441543),
 (('ATL', 'ORD'), 0.06860209458333448),
 (('ATL', 'RDU'), 0.06258510329846709),
 (('CLE', 'MSP'), 0.061839250399270865),
 (('TPA', 'JFK'), 0.061714928183888296),
 (('MEM', 'LGA'), 0.05830498985671948),
 (('CLE', 'CLT'), 0.058012699599350966),
 (('ATL', 'FLL'), 0.05785676366783496),
 (('ATL', 'MIA'), 0.05635017917925836),
 (('MDW', 'STL'), 0.05628959974758388),
 (('MIA', 'LGA'), 0.054931698770609695),
 (('IAD', 'MIA'), 0.05369753496571283),
 (('TPA', 'ATL'), 0.05211765143111149),
 (('MCO', 'JFK'), 0.04699693618685988),
 (('IAH', 'PHL'), 0.046189047603391535),
 (('PHL', 'DTW'), 0.04393273346940192),
 (('IAH', 'BOS'), 0.04352586817581191),
 (('TPA', 'DCA'), 0.04303095562315109),
 (('BOS', 'MCO'), 0.0430295183088538